In [3]:
import wrds
import pandas as pd
import pytz
import datetime as dt
import pandas_market_calendars as mcal
db = wrds.Connection(wrds_username = "connorwz")

Loading library list...
Done


In [4]:
mapping_file_query = """ 
                    SELECT DISTINCT 
                    a.permno,a.comnam,b.rp_entity_id,b.entity_name
                    FROM (SELECT permno,ncusip,comnam FROM crsp.dse
                    WHERE ncusip IS NOT NULL) as a,
                    (SELECT rp_entity_id,cusip,entity_name FROM rpna.wrds_rpa_company_mappings 
                    WHERE cusip IS NOT NULL) as b
                    WHERE a.ncusip=substr(b.cusip,1,8)
"""
mapping_file = db.raw_sql(mapping_file_query)

In [5]:
mapping_file.head()

,permno,comnam,rp_entity_id,entity_name
0,10012,D P A C TECHNOLOGIES CORP,EEF6A8,DPAC Technologies Corp.
1,10026,J & J SNACK FOODS CORP,B72BE9,J&J Snack Foods Corp.
2,10028,ENVELA CORP,B9BDE8,Envela Corp.
3,10031,A A IMPORTING INC,91E312,A.A. Importing Co. Inc.
4,10032,PLEXUS CORP,A2CAF7,Plexus Corp.


In [6]:
# No permno is mapped to more than one RP entity_ids 
mapping_file.groupby("permno").filter(lambda sub:len(sub.rp_entity_id.unique()) > 1).sort_values(by = "permno")

,permno,comnam,rp_entity_id,entity_name


In [7]:
# Some RP entity_ids are mapped to more than one permnos
entity_id_mapped_morethan1 = mapping_file.groupby("rp_entity_id").filter(lambda sub:len(sub.permno.unique()) > 1).sort_values(by = "rp_entity_id")
entity_id_mapped_morethan1.head()

,permno,comnam,rp_entity_id,entity_name
494,13452,TRONOX HOLDINGS PLC,00D6B5,Tronox Holdings PLC
7941,91153,TRONOX INC,00D6B5,Tronox Holdings PLC
7969,91277,QURATE RETAIL INC Q V C GROUP,0C6861,Qurate Retail Group Inc.
7967,91277,QURATE RETAIL GROUP INC,0C6861,Qurate Retail Group Inc.
7968,91277,QURATE RETAIL INC,0C6861,Qurate Retail Group Inc.


In [8]:
# There are many reasons resulting in this; For example, "0C6861" has 2 permnos becuase of A and B share classes
entity_id_mapped_morethan1.groupby("rp_entity_id").get_group("0C6861")

,permno,comnam,rp_entity_id,entity_name
7969,91277,QURATE RETAIL INC Q V C GROUP,0C6861,Qurate Retail Group Inc.
7967,91277,QURATE RETAIL GROUP INC,0C6861,Qurate Retail Group Inc.
7968,91277,QURATE RETAIL INC,0C6861,Qurate Retail Group Inc.
7972,91278,QURATE RETAIL INC Q V C GROUP,0C6861,Qurate Retail Group Inc.
7971,91278,QURATE RETAIL INC,0C6861,Qurate Retail Group Inc.
7970,91278,QURATE RETAIL GROUP INC,0C6861,Qurate Retail Group Inc.


In [9]:
# For "16AD58", it has 2 permnos because it changed its primary listing from the London Stock Exchange (LSE) to the New York Stock Exchange (NYSE) on September 25, 2023
entity_id_mapped_morethan1.groupby("rp_entity_id").get_group("16AD58")

,permno,comnam,rp_entity_id,entity_name
4143,24294,C R H PLC NEW,16AD58,CRH PLC
5087,75650,C R H PLC,16AD58,CRH PLC


In [10]:
# All SP500 companies between 2000 and 2023
sp_500_20_cen_query = f"""SELECT distinct permno FROM crsp.dsp500list
                        WHERE ending >= '01/01/2000' and start <= '12/31/2023'
                """
sp_500_20_cen = db.raw_sql(sp_500_20_cen_query)

In [11]:
# No SP500 companies are mapped to more than one RP entity_id so we are free to use this mapping file 
# if analysis are restricted to SP500 companies
pd.Series(entity_id_mapped_morethan1.permno.unique()).isin(sp_500_20_cen).sum()

0

In [23]:
mapping_file = mapping_file[["permno","rp_entity_id"]]
mapping_file.drop_duplicates(inplace=True)
mapping_file.to_csv("Mapping_file.csv",index=False)